In [ ]:
# ARIMA Model
model = ARIMA(air.AirPassengers, order=(1,1,2))
model_fit = model.fit(disp=0)
print(model_fit.summary())

### 2.1.5 ARIMA Model Diagnostic

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2, figsize=(18,3))
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

In [ ]:
# Actual vs Fitted
fig, ax = plt.subplots(figsize=(18,6))
model_fit.plot_predict(dynamic=False, ax=ax)
plt.legend(["Forecast","Actual"])
plt.show()

In [ ]:
# Build ARIMA Model  
model = ARIMA(train, order=(1, 1, 2))  
fitted = model.fit(disp=-1)  

# ARIMA Forecast
fc, se, conf = fitted.forecast(12, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(18,6), dpi=100)
plt.plot(train, label='Training')
plt.plot(test, label='Actual')
plt.plot(fc_series, label='Forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('ARIMA Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

### 2.1.6 ARIMA Performance Metrics

In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mae = np.mean(np.abs(forecast - actual))  # MAE
    mse = np.mean(np.square(forecast - actual))  # MSE
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    rmse = np.sqrt(np.mean(np.square(forecast - actual)))  # RMSE
    
    return({'mae':mae, 'mse':mse, 'mape':mape, 'rmse':rmse})


In [ ]:
# ARIMA Model Performance Metrics
arima_performance = forecast_accuracy(test.values, fc)
print("ARIMA Performance:")
print("="*24)
for k, v in arima_performance.items():
    print(k +" : "+ str(v))